<a href="https://colab.research.google.com/github/KangnamUniv-ML/Titanic-ML/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install kaggle

## 참고 레퍼런스

- [kaggle API 사용법](https://velog.io/@skyepodium/Kaggle-API-%EC%82%AC%EC%9A%A9%EB%B2%95)

In [1]:
import os

# 닉네임, 토큰 설정
# os.environ["KAGGLE_USERNAME"] = ''
# os.environ["KAGGLE_KEY"] = ""

# 데이터 받기, colab에서 리눅스 명령어를 사용할때는 앞에 !가 붙습니다.
!kaggle competitions download -c titanic

  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 4.96MB/s]


In [2]:
!unzip -o titanic.zip
!rm titanic.zip
!ls -al

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
total 108
drwxr-xr-x 7 jovyan users   224 May 18 10:41 .
drwsrwsr-x 1 jovyan users  4096 May 18 10:41 ..
-rw-r--r-- 1 jovyan users  3258 Dec 11  2019 gender_submission.csv
drwxr-xr-x 2 jovyan users    64 May 18 10:41 .ipynb_checkpoints
-rw-r--r-- 1 jovyan users 28629 Dec 11  2019 test.csv
-rw-r--r-- 1 jovyan users  6001 May 18 10:17 Titanic.ipynb
-rw-r--r-- 1 jovyan users 61194 Dec 11  2019 train.csv


In [3]:
!kaggle competitions submit -c titanic -f gender_submission.csv -m "Subtit with Kaggle API"

100%|████████████████████████████████████████| 3.18k/3.18k [00:03<00:00, 822B/s]
Successfully submitted to Titanic - Machine Learning from Disaster